In [54]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, MaxPool2D
import cv2

In [55]:
import pandas as pd

# Chemin vers le répertoire principal des données
data_dir = 'C:/Users/mariem hssn/OneDrive/Bureau/AIProject/archive (1)/'

# Nom du fichier CSV
file_name = 'Crop_details.csv'

# Chemin complet du fichier CSV
file_path = data_dir + file_name

# Charger le fichier CSV
df = pd.read_csv(file_path)

# Afficher les premières lignes du DataFrame
df.head()


,Unnamed: 0,path,crop,croplabel
0,0,/kaggle/input/kag2/sugarcane/sugarcane037ahs.jpeg,sugarcane,3
1,1,/kaggle/input/kag2/sugarcane/sugarcane034a.jpeg,sugarcane,3
2,2,/kaggle/input/kag2/sugarcane/sugarcane032arot....,sugarcane,3
3,3,/kaggle/input/kag2/sugarcane/sugarcane036ahs.jpeg,sugarcane,3
4,4,/kaggle/input/kag2/sugarcane/sugarcane023ahs.jpeg,sugarcane,3


In [56]:
categories = os.listdir("C:/Users/mariem hssn/OneDrive/Bureau/AIProject/archive (1)/kag2")
categories

['jute', 'maize', 'rice', 'sugarcane', 'wheat']

In [57]:
getData = ImageDataGenerator(rescale=1/255,
                             shear_range=0.3,
                             horizontal_flip=True,
                            vertical_flip= True,
                            rotation_range=90)

In [58]:
kag2_data = getData.flow_from_directory("C:/Users/mariem hssn/OneDrive/Bureau/AIProject/archive (1)/kag2",
                                       target_size=(224,224),
                                       class_mode='categorical',
                                       shuffle = True,
                                       batch_size=32)

Found 804 images belonging to 5 classes.


In [63]:
model = Sequential()

model.add(Dense(512,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(5,activation="softmax"))

In [64]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',
             metrics=['accuracy'])

In [65]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
num_train_samples = 804
batch_size = 64
epochs = 25

final_model = model.fit(kag2_data,
                        steps_per_epoch=num_train_samples//batch_size,
                        epochs=epochs)


In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255)
crop_data = test_datagen.flow_from_directory("../input/agriculture-crop-images/crop_images",
                                       target_size=(224,224),
                                       class_mode='categorical',
                                       shuffle = True,
                                       batch_size=32)

In [ ]:
plt.plot(final_model.history['loss'],label='train_loss')
plt.plot(final_model.history['accuracy'],label='train_acc')
plt.legend()
plt.show()

In [ ]:
class_labels = crop_data.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())
Y_pred = model.predict_generator(crop_data)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
def predict_crop(path,actual,class_labels):
    img = cv2.imread(path)
    img = cv2.resize(img,(224,224))
    img = np.array(img).reshape((1,224,224,3))
    Y_pred = model.predict(img)
    y_pred = np.argmax(Y_pred,axis=1)
    if y_pred == actual:
        print("Correct Prediction 🎉🎊🎉")
    else:
        print("Wrong Prediction 💥💥💥")
    print('Actual class "{0}" and predicted class "{1}"'.format(class_labels[int(y_pred)],class_labels[actual]))

In [ ]:
predict_crop('/kaggle/input/agriculture-crop-images/kag2/rice/rice024ahs.jpeg',2,class_labels)
